In [24]:
# Step to ensure that the venv is being used for the project not local copies, should point at .venv in project.
import sys, shutil
print("python:", sys.executable)
print("uv:", shutil.which("uv")) 

python: c:\Users\AndrzejPytel\source\Hackathon-2025-AP-Fork\.venv\Scripts\python.exe
uv: c:\Users\AndrzejPytel\source\Hackathon-2025-AP-Fork\.venv\Scripts\uv.EXE


In [25]:
# installs into the current Jupyter kernel environment
%pip install -U uv 
#! to run shell commands
!uv pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


Using Python 3.13.7 environment at: c:\Users\AndrzejPytel\source\Hackathon-2025-AP-Fork\.venv
Audited 13 packages in 23ms


In [26]:
# LangChain + MCP Setup for Cinema Booking (HTTP-based for Jupyter)
import os
from dotenv import load_dotenv
import asyncio
import json
import requests
from typing import Dict, Any, List

# LangChain imports
from langchain_openai import ChatOpenAI, AzureChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationBufferMemory

# Official MCP adapter imports for HTTP transport
from langchain_mcp_adapters.client import MultiServerMCPClient
from langchain_mcp_adapters.tools import load_mcp_tools

# Load environment variables
load_dotenv()

print("✅ Updated imports with official MCP adapter loaded successfully!")

✅ Updated imports with official MCP adapter loaded successfully!


In [27]:
# MCP Client Setup using Official Adapter with HTTP Transport
import subprocess
import time

# Global MCP client for HTTP
mcp_client = None

async def create_mcp_tools():
    """Create MCP tools using the official LangChain MCP adapter with HTTP transport"""
    global mcp_client
    
    try:
        # Create MultiServerMCPClient with streamable_http transport for cinema and movie reviews
        mcp_client = MultiServerMCPClient({
            "cinema": {
                "transport": "streamable_http",
                "url": os.getenv("CINEMA_MCP_URL")  # Cinema MCP server with /mcp/ path
            },
            "movie_reviews": {
                "transport": "streamable_http",
                "url": os.getenv("MOVIES_MCP_URL")  # Movie Reviews MCP server
            },
        })
        
        # Get tools from the MCP servers
        tools = await mcp_client.get_tools()
        print(f"Loaded {len(tools)} MCP tools: {[tool.name for tool in tools]}")
        return tools
        
    except Exception as e:
        print(f"Error connecting to MCP HTTP servers: {e}")
        print("Make sure the MCP servers are running:")
        print("Cinema: cd src/mcp/cinema-mcp && uv run main.py (port 8010)")
        print("Movie Reviews: cd src/mcp/movie-reviews-mcp && uv run main.py (port 8011)")
        return []

print("🔗 Cinema & Movie Reviews MCP HTTP adapter setup ready!")

🔗 Cinema & Movie Reviews MCP HTTP adapter setup ready!


In [28]:
# Load MCP Tools using Official Adapter
# The tools will be loaded dynamically when setting up the agent
# No need to manually create tool wrappers - the adapter handles this automatically
print("🛠️ Ready to load Cinema & Movie Reviews MCP tools via official adapter!")

🛠️ Ready to load Cinema & Movie Reviews MCP tools via official adapter!


In [29]:
async def setup_agent():
    """Setup LangChain agent with Cinema and Movie Reviews MCP tools using official adapter"""
    
    # Initialize LLM for Azure OpenAI
    # can get this from Azure Open AI service -> Azure AI Foundry Portal
    from langchain_openai import AzureChatOpenAI
    
    llm = AzureChatOpenAI(
        deployment_name=os.getenv("DEPLOYMENT_NAME"),  # Your Azure deployment name
        api_key=os.getenv("AZURE_OPENAI_API_KEY"),
        azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"), 
        api_version=os.getenv("AZURE_API_VERSION"), 
        temperature=1
    )
    
    # Load MCP tools using official adapter
    tools = await create_mcp_tools()
    
    if not tools:
        print("No MCP tools loaded. Make sure the Cinema and Movie Reviews MCP servers are accessible.")
        return None
    
    print(f"Loaded {len(tools)} MCP tools: {[tool.name for tool in tools]}")
    
    # Create system prompt for cinema and movie reviews assistant
    system_prompt = """You are a helpful movie and cinema assistant that can help users discover movies, get reviews, and make reservations.
    
    You have access to both cinema and movie reviews MCP tools including:
    
    Cinema tools:
    - Getting current movies playing with showtimes and availability
    - Searching for movies by title, genre, date, room, and seat availability
    - Getting detailed information about specific movie showings
    - Making movie reservations for customers
    - Viewing customer reservations
    - Canceling reservations
    
    Movie reviews tools:
    - Getting detailed movie information including plot, cast, ratings
    - Searching for movies across different databases
    - Getting movie reviews and ratings from various sources
    - Getting movie genres and categories
    - Finding random movie recommendations
    
    When helping users:
    1. Use movie reviews tools to get detailed movie information, ratings, and reviews
    2. Use cinema tools to check showtimes and availability
    3. For reservations, always collect: customer name, email, number of seats wanted
    4. Use make_reservation with the exact movie details from search results
    5. Provide comprehensive movie information including reviews and cinema availability
    6. Be helpful and guide users through both movie discovery and booking process
    
    Always be friendly and provide rich information about movies including reviews, ratings, and booking options."""
    
    # Create prompt template
    prompt = ChatPromptTemplate.from_messages([
        ("system", system_prompt),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ])
    
    # Create agent
    agent = create_tool_calling_agent(llm, tools, prompt)

    # memory
    memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

    # Create agent executor with tool logging callback and verbose output
    agent_executor = AgentExecutor(agent=agent, tools=tools, memory=memory, verbose=True)
    
    return agent_executor

# Initialize the agent (now async)
agent_executor = None
print("🤖 Cinema & Movie Reviews agent setup function ready! Run the next cell to initialize.")

🤖 Cinema & Movie Reviews agent setup function ready! Run the next cell to initialize.


In [30]:
# Initialize the agent with Cinema and Movie Reviews MCP tools
async def initialize_agent():
    """Initialize the agent with Cinema and Movie Reviews MCP tools"""
    global agent_executor
    print("Initializing cinema and movie reviews agent with MCP tools...")
    agent_executor = await setup_agent()
    if agent_executor:
        print("LangChain cinema and movie reviews agent with MCP tools ready!")
    else:
        print("Failed to initialize agent. Check Cinema and Movie Reviews MCP server connections.")

# Run the initialization
await initialize_agent()

Initializing cinema and movie reviews agent with MCP tools...


Loaded 14 MCP tools: ['get_current_movies', 'get_movie_details', 'search_movies', 'make_reservation', 'get_my_reservations', 'cancel_reservation', 'get_all_reservations', 'acknowledge_reservation', 'revoke_reservation', 'get_movie_details', 'search_movies', 'get_random_movie', 'search_and_format_movies', 'get_movie_reviews_by_title']
Loaded 14 MCP tools: ['get_current_movies', 'get_movie_details', 'search_movies', 'make_reservation', 'get_my_reservations', 'cancel_reservation', 'get_all_reservations', 'acknowledge_reservation', 'revoke_reservation', 'get_movie_details', 'search_movies', 'get_random_movie', 'search_and_format_movies', 'get_movie_reviews_by_title']
LangChain cinema and movie reviews agent with MCP tools ready!


C:\Users\AndrzejPytel\AppData\Local\Temp\ipykernel_38108\1472502583.py:67: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)


In [31]:
# User Input Handler + logged agent steps
async def process_user_input(user_input: str) -> str:
    """Process user input and return LLM response using Cinema and Movie Reviews MCP tools"""
    if not agent_executor:
        return "Agent not initialized. Please run the initialization cell first."
    
    try:
        # Use the agent to process the input and get intermediate steps
        result = await agent_executor.ainvoke({"input": user_input})
        output = result.get("output") or result.get("final_output") or ""

        # Print intermediate steps if present
        steps = result.get("intermediate_steps") or []
        for step in steps:
            action = None
            observation = None
            if isinstance(step, tuple) and len(step) == 2:
                action, observation = step
            elif isinstance(step, dict) and "action" in step:
                action = step.get("action")
                observation = step.get("observation")
            else:
                continue

            tool_name = getattr(action, "tool", getattr(action, "tool_name", "unknown"))
            tool_args = getattr(action, "tool_input", getattr(action, "input", None))
            print(f"\n--- Tool: {tool_name}")
            print(f"args: {tool_args}")
            if observation is not None:
                print(f"result: {observation}")
            print("---\n")

        return output
    except Exception as e:
        return f"Error processing request: {str(e)}"

# Interactive function for easy testing
async def ask_assistant(question: str):
    """Easy-to-use function for asking the cinema and movie reviews assistant"""
    print(f"🎬 User: {question}")
    print("🤖 Assistant:")
    
    response = await process_user_input(question)
    print(response)
    return response

print("💬 Cinema and Movie Reviews user input handler ready!")

💬 Cinema and Movie Reviews user input handler ready!


In [32]:
# Test Cinema and Movie Reviews MCP server connectivity and tools
async def test_mcp_connection():
    """Test Cinema and Movie Reviews MCP server connections and list available tools"""
    tools = await create_mcp_tools()
    if tools:
        print(f"Cinema and Movie Reviews MCP HTTP servers connected successfully!")
        print(f"Available tools: {[tool.name for tool in tools]}")
        for tool in tools:
            print(f"  - {tool.name}: {tool.description}")
    else:
        print("Failed to connect to MCP HTTP servers")
        print("Make sure to start both MCP servers:")
        print("Cinema: cd src/mcp/cinema-mcp && uv run main.py (port 8010)")
        print("Movie Reviews: cd src/mcp/movie-reviews-mcp && uv run main.py (port 8011)")

# Test MCP HTTP connections
await test_mcp_connection()

Loaded 14 MCP tools: ['get_current_movies', 'get_movie_details', 'search_movies', 'make_reservation', 'get_my_reservations', 'cancel_reservation', 'get_all_reservations', 'acknowledge_reservation', 'revoke_reservation', 'get_movie_details', 'search_movies', 'get_random_movie', 'search_and_format_movies', 'get_movie_reviews_by_title']
Cinema and Movie Reviews MCP HTTP servers connected successfully!
Available tools: ['get_current_movies', 'get_movie_details', 'search_movies', 'make_reservation', 'get_my_reservations', 'cancel_reservation', 'get_all_reservations', 'acknowledge_reservation', 'revoke_reservation', 'get_movie_details', 'search_movies', 'get_random_movie', 'search_and_format_movies', 'get_movie_reviews_by_title']
  - get_current_movies: Get all currently playing movies with their showtimes and availability

Returns:
    List of all movies currently being shown with details including:
    - Movie title, description, and basic info
    - Showtimes and theater room assignments 

In [33]:
# Detailed debug of MCP connections
import traceback
import httpx

async def debug_mcp_connection():
    """Debug the Cinema and Movie Reviews MCP connections with detailed error info"""
    print("🔍 Debugging Cinema and Movie Reviews MCP connections...")
    
    # Test basic HTTP connectivity for both servers
    servers = [
        ("Cinema", "http://localhost:8010"),
        ("Movie Reviews", "http://localhost:8011")
    ]
    
    for server_name, url in servers:
        try:
            async with httpx.AsyncClient() as client:
                response = await client.get(url, timeout=5.0)
                print(f"✅ {server_name} HTTP connection works: {response.status_code}")
        except Exception as e:
            print(f"❌ {server_name} HTTP connection failed: {e}")
    
    # Test MCP client creation with detailed error tracking
    try:
        from langchain_mcp_adapters.client import MultiServerMCPClient
        
        client = MultiServerMCPClient({
            "cinema": {
                "transport": "streamable_http",
                "url": os.getenv("CINEMA_MCP_URL")
            },
            "movie_reviews": {
                "transport": "streamable_http",
                "url": os.getenv("MOVIES_MCP_URL")
            }
        })
        
        print("✅ MCP client created successfully")
        
        # Test getting tools with full error details
        try:
            tools = await client.get_tools()
            print(f"✅ Got {len(tools)} tools: {[tool.name for tool in tools]}")
            
            # Test closing connection
            await client.close()
            print("✅ Connection closed successfully")
            
        except Exception as e:
            print(f"❌ Error getting tools: {e}")
            print("Full traceback:")
            traceback.print_exc()
            
    except Exception as e:
        print(f"❌ Error creating MCP client: {e}")
        print("Full traceback:")
        traceback.print_exc()

# Run detailed debug
await debug_mcp_connection()

🔍 Debugging Cinema and Movie Reviews MCP connections...
✅ Cinema HTTP connection works: 404
✅ Cinema HTTP connection works: 404
✅ Movie Reviews HTTP connection works: 404
✅ MCP client created successfully
✅ Movie Reviews HTTP connection works: 404
✅ MCP client created successfully
✅ Got 14 tools: ['get_current_movies', 'get_movie_details', 'search_movies', 'make_reservation', 'get_my_reservations', 'cancel_reservation', 'get_all_reservations', 'acknowledge_reservation', 'revoke_reservation', 'get_movie_details', 'search_movies', 'get_random_movie', 'search_and_format_movies', 'get_movie_reviews_by_title']
❌ Error getting tools: 'MultiServerMCPClient' object has no attribute 'close'
Full traceback:
✅ Got 14 tools: ['get_current_movies', 'get_movie_details', 'search_movies', 'make_reservation', 'get_my_reservations', 'cancel_reservation', 'get_all_reservations', 'acknowledge_reservation', 'revoke_reservation', 'get_movie_details', 'search_movies', 'get_random_movie', 'search_and_format_m

Traceback (most recent call last):
  File "C:\Users\AndrzejPytel\AppData\Local\Temp\ipykernel_38108\2853036564.py", line 46, in debug_mcp_connection
    await client.close()
          ^^^^^^^^^^^^
AttributeError: 'MultiServerMCPClient' object has no attribute 'close'


In [34]:
# 🚀 EXAMPLE USAGE - Run this cell after setting up your API key!

# Simple question about current movies
await ask_assistant("What movies are currently playing?")
print("\n" + "="*50 + "\n")

# Example 2: Search for specific type of movie
# await ask_assistant("I'm looking for action movies playing tomorrow. What do you have?")
# print("\n" + "="*50 + "\n")

🎬 User: What movies are currently playing?
🤖 Assistant:


> Entering new AgentExecutor chain...

Invoking: `get_current_movies` with `{}`



Invoking: `get_current_movies` with `{}`


{
  "cinema_name": "MovieMagic Cinema",
  "total_movies": 8,
  "movies": [
    {
      "title": "Galactic Adventures",
      "description": "An epic space adventure following a crew of explorers as they journey through distant galaxies to save humanity from an alien threat.",
      "date": "2025-09-25",
      "time": "14:30",
      "room": "Theater A",
      "seats_remaining": 105,
      "seats_total": 150,
      "duration_minutes": 142,
      "genre": "Science Fiction",
      "rating": "PG-13",
      "price_per_seat": 12.5,
      "director": "Sarah Johnson",
      "cast": [
        "Alex Thompson",
        "Maria Rodriguez",
        "James Chen"
      ],
      "is_sold_out": false,
      "occupancy_percentage": 30.0
    },
    {
      "title": "The Midnight Mystery",
      "description": "A thrilling det

In [ ]:
# Interactive chat loop — keep asking questions until you exit

# Try these example questions:
# - "What movies are playing today?"
# - "Show me action movies and their reviews"
# - "Tell me about Avatar - plot, cast, and reviews"
# - "I want to see Avatar tomorrow evening - show me reviews and showtimes"
# - "Get me details for Avatar on 2025-09-25 at 19:30 in theater_a"
# - "Book 2 seats for Avatar on 2025-09-25 at 19:30 in theater_a for John Doe, john@email.com"
# - "Show my reservations for john@email.com"
# - "Find me a random movie recommendation with good reviews"

async def chat_loop():
    print("🎬 Welcome to MovieMagic Cinema & Reviews Assistant!")
    print("I can help you find movies, check reviews, get ratings, check showtimes, and make reservations.")
    print("Type 'exit' to quit. Press Enter on an empty line to skip.\n")
    
    while True:
        try:
            question = input("You: ").strip()
        except (EOFError, KeyboardInterrupt):
            print("\nExiting.")
            break
        if not question:
            continue
        if question.lower() in ("exit", "quit", "q"):
            print("🎬 Thanks for using MovieMagic Cinema & Reviews! Goodbye!")
            break
        await ask_assistant(question)

# Start the cinema and movie reviews chat loop
await chat_loop()

🎬 Welcome to MovieMagic Cinema & Reviews Assistant!
I can help you find movies, check reviews, get ratings, check showtimes, and make reservations.
Type 'exit' to quit. Press Enter on an empty line to skip.

🎬 User: What are the reviews for The Midnight Mystery
🤖 Assistant:


> Entering new AgentExecutor chain...
🎬 User: What are the reviews for The Midnight Mystery
🤖 Assistant:


> Entering new AgentExecutor chain...

Invoking: `get_movie_reviews_by_title` with `{'title': 'The Midnight Mystery'}`



Invoking: `get_movie_reviews_by_title` with `{'title': 'The Midnight Mystery'}`


Error processing request: Error executing tool get_movie_reviews_by_title: search_movies() got an unexpected keyword argument 'title'
Error processing request: Error executing tool get_movie_reviews_by_title: search_movies() got an unexpected keyword argument 'title'
🎬 User: what movies are playing in cinema currently
🤖 Assistant:


> Entering new AgentExecutor chain...
🎬 User: what movies are playing in cine

In [ ]:
# Cleanup function for HTTP MCP clients
async def cleanup_mcp():
    """Cleanup MCP client and server resources"""
    global mcp_client
    if mcp_client:
        try:
            await mcp_client.close()
            print("Cinema and Movie Reviews MCP clients closed")
        except Exception as e:
            print(f"Warning: Error closing MCP clients: {e}")
        mcp_client = None

print("🧹 Cleanup function ready!")

🧹 Cleanup function ready!
